## Import libraries
* Need pandas for dataframe
* Need BeautifulSoup for scraping
* Need requests to request url

In [124]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [125]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url)
soup = BeautifulSoup(source.text, 'lxml')

* Initiates table, postalcode, borough, and neighborhood
* Loop goes through and adds data to list

In [126]:
table = soup.find('table')
td = table.find_all('td')
postalcode = []
borough = []
neighborhood = []
for i in range(0, len(td), 3):
    postalcode.append(td[i].text.strip())
    borough.append(td[i+1].text.strip())
    neighborhood.append(td[i+2].text.strip())

* Creates Canada DataFrame and initiates columns

In [127]:
canadaDF = pd.DataFrame(data = [postalcode, borough, neighborhood]).transpose()
canadaDF.columns = ['PostalCode', 'Borough', 'Neighborhood']
canadaDF.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


* Replaces all instances in Borough and Neighboorhood of 'Not assigned' with NaN

In [128]:
canadaDF['Borough'].replace('Not assigned', np.nan, inplace= True)
canadaDF.dropna(subset = ['Borough'], inplace = True)
canadaDF['Neighborhood'].replace('Not assigned', np.nan, inplace= True)
canadaDF.dropna(subset = ['Neighborhood'], inplace = True)

* Groups all Neighborhoods by PostalCode as well as drops duplicates

In [129]:
canadaDF['Neighborhood'] = canadaDF.groupby('PostalCode')['Neighborhood'].transform(lambda neigh: ', '.join(neigh))
canadaDF = canadaDF.drop_duplicates()
if (canadaDF.index.name != 'PostalCode'):
    canadaDF = canadaDF.set_index("PostalCode")

In [130]:
canadaDF.head()

,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [131]:
canadaDF.shape

(103, 2)

In [132]:
csv = pd.read_csv('Geospatial_Coordinates.csv')
csv.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)
df = pd.merge(canadaDF, csv, how = 'inner', on = 'PostalCode')
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [134]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import json
from pandas.io.json import json_normalize

Solving environment: ...working... done

# All requested packages already installed.



* Sorting boroughs to get only those in North York

In [135]:
NorthYork = df[df['Borough'] == 'North York'].reset_index(drop = True)
NorthYork

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073
5,M3C,North York,Don Mills,43.725900,-79.340923
6,M2H,North York,Hillcrest Village,43.803762,-79.363452
7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262


In [136]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent = 'canada')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [137]:
torontoMap = folium.Map(location = [latitude, longitude], zoom_start = 10)
for lat, lng, borough, neighborhood in zip(NorthYork['Latitude'], NorthYork['Longitude'], NorthYork['Borough'], NorthYork['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, lng], radius = 5, popup = label, color = 'blue', fill = True, fill_color = '#3186cc', fill_opacity = 0.7, parse_html = False).add_to(torontoMap)
torontoMap

#### Getting Foursquare API

In [138]:
CLIENT_ID = 'RN530BC0LUAPUIO15YIWGNE0NKN3OWUM4CE4EF5YAL1PKGK2' # your Foursquare ID
CLIENT_SECRET = 'PEFFPSDRAKNQNDMRXK5HGWUHVDFEAWTL5P4AOM11SZTMVMOO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RN530BC0LUAPUIO15YIWGNE0NKN3OWUM4CE4EF5YAL1PKGK2
CLIENT_SECRET:PEFFPSDRAKNQNDMRXK5HGWUHVDFEAWTL5P4AOM11SZTMVMOO


In [139]:
NorthYork.loc[0, 'Neighborhood']

'Parkwoods'

In [140]:
neighborhood_latitude = NorthYork.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = NorthYork.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = NorthYork.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [141]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [142]:
results = requests.get(url).json()

In [143]:
results

{'meta': {'code': 200, 'requestId': '5ffcc51a78a0a07d3b977208'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [144]:
def getCatType(row):
    try:
        catList = row['categories']
    except:
        catList = row['venue.categories']
    if len(catList) == 0:
        return None
    else:
        return catList[0]['name']

In [145]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(getCatType, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-145-7bea522fbe85>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,Bella Vita Catering & Private Chef Service,BBQ Joint,43.756651,-79.331524


In [146]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


In [147]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [148]:
NorthYorkVenues = getNearbyVenues(names = NorthYork['Neighborhood'], latitudes = NorthYork['Latitude'], longitudes = NorthYork['Longitude'])

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [149]:
print(NorthYorkVenues.shape)
NorthYorkVenues.head()

(239, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [150]:
NorthYorkVenues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Don Mills,25,25,25,25,25,25
Downsview,13,13,13,13,13,13
"Fairview, Henry Farm, Oriole",69,69,69,69,69,69
Glencairn,6,6,6,6,6,6
Hillcrest Village,5,5,5,5,5,5
Humber Summit,3,3,3,3,3,3


In [151]:
print('There are {} uniques categories.'.format(len(NorthYorkVenues['Venue Category'].unique())))

There are 104 uniques categories.


In [152]:
# one hot encoding
NYonehot = pd.get_dummies(NorthYorkVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NYonehot['Neighborhood'] = NorthYorkVenues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NYonehot.columns[-1]] + list(NYonehot.columns[:-1])
NYonehot = NYonehot[fixed_columns]
NYonehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,...,Supermarket,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [153]:
NYonehot.shape

(239, 105)

In [154]:
NYgrouped = NYonehot.groupby('Neighborhood').mean().reset_index()
NYgrouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,...,Supermarket,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.047619,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.045455,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.045455,0.000000,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000
3,Don Mills,0.000000,0.000000,0.000000,0.04,0.000000,0.040000,0.040000,0.000000,0.000000,...,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Downsview,0.000000,0.076923,0.000000,0.00,0.000000,0.000000,0.076923,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Fairview, Henry Farm, Oriole",0.000000,0.000000,0.014493,0.00,0.000000,0.014493,0.000000,0.000000,0.028986,...,0.000000,0.000000,0.014493,0.014493,0.000000,0.014493,0.014493,0.014493,0.000000,0.014493
6,Glencairn,0.000000,0.000000,0.000000,0.00,0.000000,0.166667,0.000000,0.000000,0.166667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Hillcrest Village,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Humber Summit,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Humberlea, Emery",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [155]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [156]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NYgrouped['Neighborhood']

for ind in np.arange(NYgrouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NYgrouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Middle Eastern Restaurant,Deli / Bodega,Intersection,Park,Pet Store,Pharmacy,Pizza Place,Bridal Shop
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Distribution Center,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Butcher,Grocery Store,Liquor Store,Café,Juice Bar,Indian Restaurant,Pharmacy
3,Don Mills,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Restaurant,Sandwich Place,Clothing Store,Chinese Restaurant,Café,Dim Sum Restaurant
4,Downsview,Grocery Store,Park,Home Service,Airport,Food Truck,Liquor Store,Athletics & Sports,Discount Store,Bank,Shopping Mall


In [157]:
# set number of clusters
kclusters = 5

NYgroupedclustering = NYgrouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NYgroupedclustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 0, 2, 2, 2, 2, 3])

In [161]:
NYmerged = NorthYork

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
NYmerged = NYmerged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

NYmerged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,BBQ Joint,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Coffee Shop,French Restaurant,Hockey Arena,Portuguese Restaurant,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2.0,Clothing Store,Athletics & Sports,Furniture / Home Store,Event Space,Coffee Shop,Vietnamese Restaurant,Boutique,Accessories Store,BBQ Joint,Bakery
3,M3B,North York,Don Mills,43.745906,-79.352188,2.0,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Restaurant,Sandwich Place,Clothing Store,Chinese Restaurant,Café,Dim Sum Restaurant
4,M6B,North York,Glencairn,43.709577,-79.445073,2.0,Park,Asian Restaurant,Pizza Place,Bakery,Pub,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store


In [165]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NYmerged['Latitude'],NYmerged['Longitude'], NYmerged['Neighborhood'], NYmerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters